# Agent Instructions

In [ ]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.3 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from agents import Agent,Runner,AsyncOpenAI,OpenAIChatCompletionsModel,set_tracing_disabled,RunResult,RunContextWrapper
from google.colab import userdata

set_tracing_disabled(True)
gemini_api_key = userdata.get('GEMINI_API_KEY')

external_client = AsyncOpenAI(
    api_key = gemini_api_key,
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
)

model = OpenAIChatCompletionsModel(
    openai_client = external_client,
    model = "gemini-2.0-flash"
)

In [ ]:
import asyncio

## Basic String Instructions

In [ ]:
agent_basic:Agent = Agent(
    name = "Basic Agent",
    instructions = "You are a helpful assistant that provides short and concise answers",
    model = model,
)

In [ ]:
async def main():
  result:RunResult = await Runner.run(agent_basic, "What is the meaning of life?")
  print(result.final_output)

asyncio.run(main())

The meaning of life is subjective and varies for each individual.



## Detailed String Agent

In [ ]:
agent_detailed:Agent = Agent(
    name = "Detailed Agent",
    instructions="""You are an expert Python developer.
    - Always provide working code examples
    - Explain your reasoning step by step
    - Keep responses under 200 words
    - Use best practices and modern Python syntax
    """,
    model = model
)

In [ ]:
async def main():
  result:RunResult =await Runner.run(agent_detailed,"Tell me about Asynchronous Programming")
  print(result.final_output)

asyncio.run(main())

Asynchronous programming allows a program to initiate a task and then continue executing other tasks without waiting for the first task to complete. This contrasts with synchronous programming, where the program waits for each task to finish before moving to the next.

Here's a breakdown:

1.  **Non-Blocking:** Asynchronous operations don't block the main thread. Instead, they allow the program to remain responsive and handle other tasks.
2.  **Concurrency:** Achieves concurrency by allowing multiple tasks to make progress seemingly simultaneously. This is especially beneficial for I/O-bound operations (e.g., network requests, file reads).
3.  **`async` and `await`:** Python uses the `async` keyword to define coroutines (functions that can be paused and resumed) and the `await` keyword to pause execution until an asynchronous operation completes.

```python
import asyncio

async def fetch_data():
    await asyncio.sleep(1)  # Simulate network delay
    return "Data fetched!"

async def

## Simple Callable Instructions

In [ ]:
# making function that gives dynamic instructions when called
def dynamic_instructions(ctx:RunContextWrapper[None],agent:Agent) -> str:
  print("RECEIVED CONTEXT : ", ctx)
  print("RECEIVED AGENT : ", agent)
  return f"You are a {agent.name}. Responed user queries shortly and concisely"

In [ ]:
agent_callable:Agent = Agent(
    name = "Dynamic Agent",
    instructions = dynamic_instructions,
    model = model
)

In [ ]:
async def main():
  result:RunResult =await Runner.run(agent_callable, "What is the meaning of life?")
  print(result.final_output)

asyncio.run(main())

RECEIVED CONTEXT :  RunContextWrapper(context=None, usage=Usage(requests=0, input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0))
RECEIVED AGENT :  Agent(name='Dynamic Agent', instructions=<function dynamic_instructions at 0x7913c56d5a80>, prompt=None, handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x7913b3cd0dd0>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)


## Context Aware Callable Instructions

In [ ]:
!pip install -Uq pydantic

In [ ]:
## making simple context just for demo usage
from pydantic import BaseModel

class Message(BaseModel):
  role:str
  content:str

class Messages(BaseModel):
  messages:list[Message]


In [ ]:
def context_aware_instructions(ctx:RunContextWrapper[Messages],agent:Agent) -> str:
  # print("RECEIVED CONTEXT : ", ctx)
  # print("RECEIVED AGENT : ", agent)

  message_count = len(ctx.context.messages)

  if message_count == 0:
      return "You are a friendly assistant. Introduce yourself and ask how you can help."
  elif message_count < 3:
      return "You are a helpful assistant. Be encouraging and detailed in your responses."
  else:
      return "You are an experienced assistant. Be concise but thorough."

In [ ]:
agent_context_aware:Agent = Agent(
    name = "Context Aware Agent",
    instructions = context_aware_instructions,
    model = model,
)

In [ ]:
async def main():
  ctx0 = Messages(messages=[])
  result1:RunResult = await Runner.run(agent_context_aware, "Hi",context = ctx0)
  print("AGENT INSTRUCTIONS:", context_aware_instructions(RunContextWrapper(ctx0), agent_context_aware))
  print(result1.final_output)

  ctx2 = Messages(messages=[Message(role="user", content="Hi"),
                            Message(role="assistant", content="Hello!")])
  result2:RunResult = await Runner.run(agent_context_aware, "Tell me about Asynchronous Programming",context = ctx2)
  print("AGENT INSTRUCTIONS:", context_aware_instructions(RunContextWrapper(ctx2), agent_context_aware))
  print(result2.final_output)

  ctx4 = Messages(messages=[Message(role="user", content="Hi"),
                            Message(role="assistant", content="Hello!"),
                            Message(role="user", content="How do I code a loop?"),
                            Message(role="assistant", content="Here’s how...")])
  result3:RunResult = await Runner.run(agent_context_aware, "Hi",context = ctx2)
  print("AGENT INSTRUCTIONS:", context_aware_instructions(RunContextWrapper(ctx4), agent_context_aware))
  print(result3.final_output)

asyncio.run(main())

AGENT INSTRUCTIONS: You are a friendly assistant. Introduce yourself and ask how you can help.
Hello! I'm a friendly AI assistant here to help you with information, creative content, and answering your questions. How can I assist you today?

AGENT INSTRUCTIONS: You are a helpful assistant. Be encouraging and detailed in your responses.
Asynchronous programming is a powerful technique that allows your program to start a potentially long-running task and continue executing other code without waiting for that task to complete. It's like ordering food at a restaurant – you place your order (start the task), and then you can chat with your friends, read a book, or do other things (continue executing code) while the kitchen prepares your meal (the task runs in the background). When your food is ready (the task is complete), the waiter brings it to you (you're notified of the completion), and you can finally eat (process the result of the task).

Here's a breakdown of why and how it works:

*

## Async Callable Instructions

In [ ]:
from datetime import datetime

In [ ]:
async def async_instructions(ctx:RunContextWrapper[None],agent:Agent):
  await asyncio.sleep(0.1)
  current_time = asyncio.get_event_loop().time()
  parsed_time = datetime.fromtimestamp(current_time)

  return f"""You are {agent.name}, an AI assistant with real-time capabilities.Current timestamp: {parsed_time}
  Provide helpful and timely responses."""

In [ ]:
agent_async:Agent = Agent(
    name = "Async Agent",
    instructions = async_instructions,
    model = model,
)

In [ ]:
async def main():
  result:RunResult =await Runner.run(agent_async, "What time it is?")
  print("AGENT INSTRUCTIONS : ",await async_instructions(RunContextWrapper(None), agent_async))
  print(result.final_output)

asyncio.run(main())

AGENT INSTRUCTIONS :  You are Async Agent, an AI assistant with real-time capabilities.Current timestamp: 1970-01-01 01:38:32.630480
  Provide helpful and timely responses.
Current time is 1970-01-01 01:38:35.492654



## Stateful Callable Instructions

In [ ]:
class instruction_generator:
  def __init__(self):
    self.interaction_count = 0

  def __call__(self,ctx:RunContextWrapper[None],agent:Agent):
    self.interaction_count+=1

    if self.interaction_count == 1:
      return "You are a learning assistant. This is our first interaction - be welcoming!"
    elif self.interaction_count <= 3:
      return f"You are a learning assistant. This is interaction #{self.interaction_count} - build on our conversation."
    else:
      return f"You are an experienced learning assistant. We've had {self.interaction_count} interactions - be efficient."

In [ ]:
instruction_gen = instruction_generator()

agent_stateful:Agent = Agent(
    name = "Stateful Agent",
    instructions = instruction_gen,
    model = model,
)

In [ ]:
async def main():
    for i in range(4):
        result = await Runner.run(
            agent_stateful, f"Question {i+1}: Tell me about Python"
        )
        print(f"[Interaction {i+1}]:", result.final_output[:100] + "...")

asyncio.run(main())

[Interaction 1]: Python is a high-level, general-purpose programming language. It emphasizes code readability and use...
[Interaction 2]: Python is a high-level, general-purpose programming language. Key features include:

*   **Readabili...
[Interaction 3]: Python is a high-level, general-purpose programming language. Key features include:

*   **Readabili...
[Interaction 4]: Python is a high-level, general-purpose programming language known for its readability and versatili...
